In [1]:
include("../src/UNSflow.jl")

UNSflow

In [7]:
using UNSflow

import UNSflow.mutual_ind

In [11]:
v1 = ThreeDVort([1;0;1;], [0.; 1.; 0.;], 0.02,[0.;0.;0.;], [0.;0.;0.;] )

UNSflow.ThreeDVort([1.0,0.0,1.0],[0.0,1.0,0.0],0.02,[0.0,0.0,0.0],[0.0,0.0,0.0])

In [13]:
v2= ThreeDVort([-1;1;0;], [0.; 1.; 0.;], 0.02,[0.;0.;0.;], [0.;0.;0.;] )

UNSflow.ThreeDVort([-1.0,1.0,0.0],[0.0,1.0,0.0],0.02,[0.0,0.0,0.0],[0.0,0.0,0.0])

In [8]:
function fgfromrhoGauss(rho::Float64)
    f = sqrt(2./pi)*exp(-rho^2/2.)
    g = erf(rho/sqrt(2.)) - rho*f
    return f, g
end

function mutual_ind(vorts::Vector{ThreeDVort})
      for i = 1:length(vorts)
          for j = i+1:length(vorts)
              dx = vorts[i].x[1] - vorts[j].x[1]
              dy = vorts[i].x[2] - vorts[j].x[2]
              dz = vorts[i].x[3] - vorts[j].x[3]
              #source - tar

              r = sqrt(dx*dx + dz*dz + dy*dy)
              rhoi = r/vorts[i].vc
              rhoj = r/vorts[i].vc

              f, g = fgfromrhoGauss(rhoi)
            
              vorts[j].v += -g*cross([dx; dy; dz],vorts[i].s)/(4.*pi*r^3)
                
              vorts[j].ds += -(-g*cross(vorts[j].s, vorts[i].s)/rhoi^3 + 
                ((3*g/rhoi^3 - f)*dot(vorts[j].s, [dx;dy;dz])*cross([dx;dy;dz],vorts[i].s))/r^2)/(4*pi*vorts[i].vc^3)  
                
            
            f, g = fgfromrhoGauss(rhoj)
            vorts[i].v += -g*cross([dx; dy; dz],vorts[j].s)/(4.*pi*r^3)
                
            vorts[i].ds += -(-g*cross(vorts[i].s, vorts[j].s)/rhoj^3 + 
            ((3*g/rhoj^3 - f)*dot(vorts[i].s, [dx;dy;dz])*cross([dx;dy;dz],vorts[j].s))/r^2)/(4*pi*vorts[j].vc^3)  
                

          end
      end
      return vorts
end

mutual_ind (generic function with 2 methods)

In [14]:
mutual_ind([v1; v2])

2-element Array{UNSflow.ThreeDVort,1}:
 UNSflow.ThreeDVort([1.0,0.0,1.0],[0.0,1.0,0.0],0.02,[0.0125322,0.0,-0.0250644],[-0.00270728,0.0,0.00541456])  
 UNSflow.ThreeDVort([-1.0,1.0,0.0],[0.0,1.0,0.0],0.02,[0.00541456,0.0,-0.0108291],[-0.00270728,0.0,0.00541456])